In [3]:
import cv2 as cv
import os
import numpy as np
import math

In [4]:
def rescaleFrame(frame, scale=0.5):
    width = int(frame.shape[1] * scale)
    height= int(frame.shape[0] * scale)
    dims  = (width,height)
    
    return cv.resize(frame, dims, interpolation=cv.INTER_AREA)

In [5]:
def reizeFrame(frame, width, height):
    dims  = (width,height)
    
    return cv.resize(frame, dims, interpolation=cv.INTER_AREA)

In [6]:
def rotateImage(image, angle):
    h, w = image.shape[:2]
    img_c = (w / 2, h / 2)

    rot = cv.getRotationMatrix2D(img_c, angle, 1)

    rad = math.radians(angle)
    sin = math.sin(rad)
    cos = math.cos(rad)
    b_w = int((h * abs(sin)) + (w * abs(cos)))
    b_h = int((h * abs(cos)) + (w * abs(sin)))

    rot[0, 2] += ((b_w / 2) - img_c[0])
    rot[1, 2] += ((b_h / 2) - img_c[1])

    outImg = cv.warpAffine(image, rot, (b_w, b_h), flags=cv.INTER_LINEAR)
    return outImg

In [7]:
def crop_img(img, scale=1.0, right=0, down=0):
    center_x, center_y = (img.shape[1] / 2) + right, (img.shape[0] / 2) + down
    width_scaled, height_scaled = img.shape[1] * scale, img.shape[0] * scale
    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
    img_cropped = img[int(top_y):int(bottom_y)-200, int(left_x):int(right_x)]
    return img_cropped

def crop_sc(img, scale=1.0, right=0, down=0):
    center_x, center_y = (img.shape[1] / 2) + right, (img.shape[0] / 2) + down
    width_scaled, height_scaled = img.shape[1] * 0.75, img.shape[0] * scale
    left_x, right_x = center_x - width_scaled / 2, center_x + width_scaled / 2
    top_y, bottom_y = center_y - height_scaled / 2, center_y + height_scaled / 2
    img_cropped = img[int(top_y):int(bottom_y)-200, int(left_x):int(right_x)-200]
    return img_cropped

In [8]:
refPt = []
endPt = []
scope = None
draw  = False
clone = None
w2=240
h2=135

In [9]:
input_dir='imgs/to_label/'
data_dir='data/'
count_file='count.txt'
label_file='label.csv'

In [10]:
def click_and_crop(event, x, y, flags, param):    
    global refPt, draw, w2, h2, Origf, clone, scope, frame, endPt

    if event == cv.EVENT_RBUTTONDOWN:
        frame = clone.copy()
        if y+h2*2> frame.shape[0]: y=frame.shape[0]-h2*2
        if x+w2*2> frame.shape[1]: x=frame.shape[1]-w2*2
        scope = (x,y)
        
        cv.rectangle(frame, (x,y), (x+w2*2,y+h2*2), (255, 0, 0), 2)        
        cv.imshow("image", frame)
    
    if event == cv.EVENT_LBUTTONDOWN:
        if draw == False:
            refPt = [x, y]
            endPt = []
        draw  = True
            
    elif event == cv.EVENT_MOUSEMOVE and draw:
        frame = clone.copy()
        if scope != None:
            cv.rectangle(frame, scope, (scope[0]+w2*2,scope[1]+h2*2), (255, 0, 0), 2)        
        if abs(refPt[0]-x)>(w2*2)-40: refPt[0]=abs(x-((w2*2)-40)*int((x-refPt[0])/abs(refPt[0]-x)))
        if abs(refPt[1]-y)>(h2*2)-20: refPt[1]=abs(y-((h2*2)-20)*int((y-refPt[1])/abs(refPt[1]-y)))
            
        cv.rectangle(frame, (refPt[0],refPt[1]), (x,y), (0, 255, 0), 2)
        cv.imshow("image", frame)
        
        
    elif event == cv.EVENT_LBUTTONUP and draw:
        
        frame = clone.copy()  
        endPt = [x, y]
        
        x1,y1,x2,y2=refPt+endPt
        if x2<x1: 
            refPt[0]=x2
            endPt[0]=x1
        if y2<y1: 
            refPt[1]=y2
            endPt[1]=y1
        
        if scope == None: 
            x,y = int((refPt[0]+x)/2)-w2-20, int((refPt[1]+y)/2)-h2-10
            if y+h2*2> frame.shape[0]: y=frame.shape[0]-h2*2
            if x+w2*2> frame.shape[1]: x=frame.shape[1]-w2*2
            if x<0: x = 0
            if y<0: y = 0
            scope = (x,y)
        #elif endPt[0] > scope[0]+(w2*2)-20 or endPt[1]>scope[1]+(h2*2)-10:
        #    scope = (endPt[0]-(w2*2)+20, endPt[1]-(h2*2)+10)
            
        if refPt[0]<scope[0]+20: refPt[0]=scope[0]+20
        if refPt[1]<scope[1]+10: refPt[1]=scope[1]+10 
        if endPt[0]>scope[0]+(w2*2)-20: endPt[0]=scope[0]+(w2*2)-20 
        if endPt[1]>scope[1]+(h2*2)-10: endPt[1]=scope[1]+(h2*2)-10     
        
        cv.rectangle(frame, scope, (scope[0]+w2*2,scope[1]+h2*2), (255, 0, 0), 2) 
            
        #if refPt[0]-20<scope[0]: refPt[0]=scope[0]+20
        #if endPt[0]+20>scope[0]+(w2*2): endPt[0]=scope[0]-20+(w2*2)
        #if refPt[1]-10<scope[1]: refPt[1]=scope[1]+10
        #if endPt[1]+10>scope[1]+(h2*2): endPt[1]=scope[1]-10+(h2*2)
        
        cv.rectangle(frame, (refPt[0],refPt[1]), (endPt[0],endPt[1]), (0, 255, 0), 2)
        cv.imshow("image", frame)
        
        draw = False
        

In [11]:
def label(event, x, y, flags, param):    
    global refPt, draw, w2, h2, Origf, clone, scope, frame, endPt

    if event == cv.EVENT_LBUTTONDOWN:
        if draw == False:
            refPt = [x, y]
            endPt = []
        draw  = True
            
    elif event == cv.EVENT_MOUSEMOVE and draw:
        frame = clone.copy()
        if abs(refPt[0]-x)>(w2*2)-40: refPt[0]=abs(x-((w2*2)-40)*int((x-refPt[0])/abs(refPt[0]-x)))
        if abs(refPt[1]-y)>(h2*2)-20: refPt[1]=abs(y-((h2*2)-20)*int((y-refPt[1])/abs(refPt[1]-y)))
            
        cv.rectangle(frame, (refPt[0],refPt[1]), (x,y), (0, 255, 0), 2)
        cv.imshow("image", frame)
        
        
    elif event == cv.EVENT_LBUTTONUP and draw:
        
        frame = clone.copy()  
        endPt = [x, y]
        
        x1,y1,x2,y2=refPt+endPt
        if x2<x1: 
            refPt[0]=x2
            endPt[0]=x1
        if y2<y1: 
            refPt[1]=y2
            endPt[1]=y1
        
            
        if refPt[0]<scope[0]+20: refPt[0]=scope[0]+20
        if refPt[1]<scope[1]+10: refPt[1]=scope[1]+10 
        if endPt[0]>scope[0]+(w2*2)-20: endPt[0]=scope[0]+(w2*2)-20 
        if endPt[1]>scope[1]+(h2*2)-10: endPt[1]=scope[1]+(h2*2)-10     
        
        cv.rectangle(frame, (refPt[0],refPt[1]), (endPt[0],endPt[1]), (0, 255, 0), 2)
        cv.imshow("image", frame)
        
        draw = False
        

In [33]:
with open(data_dir+count_file,'r') as f:
    counters = f.read().split(',')
    cc = int(counters[0])
    nc = int(counters[1])
    
print(cc,nc)

2810 2556


In [32]:
with open(data_dir+count_file,'r') as f:
    counters = f.read().split(',')
    cc = int(counters[0])
    nc = int(counters[1])
    
refPt = []
endPt = []
scope = None
draw  = False
clone = None

cont = True
cr=[]
pr=30

#with open('data/label.csv','w') as f:
#    pass

for file in os.listdir(input_dir):
    if file.lower().endswith(".jpg") and cont:
        #print(file)
        frame = np.real(cv.imread(input_dir+file))
        #frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        scope=[0,0]
        
        frame = rotateImage(frame,0)  
        frame = cv.resize(frame, (w2*2, h2*2), interpolation=cv.INTER_AREA)
        clone = frame.copy()
        
        cv.namedWindow("image")
        cv.setMouseCallback("image", label)
            
        while True:
                    cv.imshow('image', frame)

                    key = cv.waitKey(0) & 0xFF
                    
                    if key == ord("s"):
                            
                            scope = [0,0]
                            roi = clone[scope[1]:scope[1]+h2*2, scope[0]:scope[0]+w2*2]
                            
                            with open(data_dir+label_file,'a+') as f:
                                if len(refPt)==2 and len(endPt)==2:
                                    f.write(data_dir+'c/'+str(cc)+'.jpg,'+','.join([str(elem) for elem in refPt+endPt])+'\n')
                                    cv.imwrite(data_dir+'c/'+str(cc)+'.jpg', roi)
                                    cc+=1
                                elif scope == [0,0]:
                                    cv.imwrite(data_dir+'n/'+str(nc)+'.jpg', roi)
                                    f.write(data_dir+'n/'+str(nc)+'.jpg,'+'0,0,0,0\n')
                                    nc+=1
                            frame = clone.copy()
                            scope = [0,0]
                            refPt = []
                            endPt = []
                            break
                    
                    if key == ord("c"):
                        frame = clone.copy()
                    
                    if key == ord("q") or key == 27:
                        cont=False
                        cv.destroyAllWindows()
                        break
                    
                    if key == ord("x"):
                        break
                    
                    if key == ord("z"):
                        break
        os.remove(f"{input_dir}{file}")
        
with open(data_dir+count_file,'w') as f:
    f.write(str(cc)+","+str(nc))
    
cv.destroyAllWindows()
